In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/central-proryv/submission.csv
/kaggle/input/central-proryv/train.csv
/kaggle/input/central-proryv/test.csv


In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')

import fasttext as ft

from sklearn.manifold import TSNE
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest

In [3]:
path_to_train = '/kaggle/input/central-proryv/train.csv'
path_to_test = '/kaggle/input/central-proryv/test.csv'

In [4]:
train = pd.read_csv(path_to_train)
test = pd.read_csv(path_to_test)

In [5]:
train = train.rename(columns={'Решение             ': 'Решение'})
test = test.rename(columns={'Решение             ': 'Решение'})

In [6]:
new_names = {'Содержание':'content', 'Сервис':'service', 'Приоритет':'priority', 
             'Статус': 'status', 'Функциональная группа':'group', 'Крайний срок':'deadline', 
             'Дата обращения':'application_date', 'Тип обращения на момент подачи':'original_type',
             'Тип обращения итоговый':'final_type', 'Решение':'decision', 'Тип переклассификации':'target',
             'Дата восстановления':'repair_date', 'Дата закрытия обращения':'closing_date',
             'Критичность':'criticality', 'Влияние':'influence', 'Система':'system', 'Место':'place'}

In [7]:
train = train.rename(columns=new_names)
train = train.dropna(subset=['content'])

train.reset_index(drop=True, inplace=True)
ex_train = train.copy()

In [8]:
import re   #text preprocessing

def remove_punctuation(text):
    # В регулярном выражении \w соответствует любому словесному символу (это то же самое, что [a-zA-Z0-9_]).
    # + указывает на то, что данное выражение должно совпадать один или более раз. 
    # \s соответствует любому пробельному символу; это эквивалентно классу [ \t\n\r\f\v].
    # Таким образом, \w+|\s будет искать либо слова, либо пробельные символы.
    text_lower = text.lower()
    text_without_n = re.sub('\n', ' ', text_lower)
    result = re.findall(r'\w+|\s', text_without_n)
    return ''.join(result).strip()

In [9]:
train['content'] = train['content'].apply(remove_punctuation)

In [11]:
with open('/kaggle/working/train_cont.txt', 'w', encoding='utf-8') as f:
    for each_text in train['content']:
        f.writelines(f'{each_text}\n')

In [12]:
model_cont = ft.train_unsupervised('/kaggle/working/train_cont.txt')

Read 0M words
Number of words:  452
Number of labels: 0
Progress: 100.0% words/sec/thread:  300506 lr:  0.000000 avg.loss:  2.511150 ETA:   0h 0m 0s


In [13]:
train['cont'] = train['content'].apply(model_cont.get_sentence_vector)
emb_train_cont = train[['cont']]

emb_train_cont = pd.DataFrame(emb_train_cont['cont'].tolist())

In [14]:
display(emb_train_cont)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.147142,0.151215,-0.212896,-0.109709,0.167062,-0.013065,0.140043,0.092434,0.087104,0.181758,...,0.088995,0.047377,0.188399,-0.197008,0.063316,-0.157430,0.031188,0.152727,-0.030149,-0.027822
1,-0.154199,0.151116,-0.222792,-0.113196,0.168926,-0.009871,0.125979,0.084657,0.102202,0.176484,...,0.087816,0.031035,0.193004,-0.197343,0.056148,-0.168462,0.026245,0.138969,-0.014906,-0.023391
2,-0.088002,0.059959,-0.139508,-0.053796,0.141902,0.005851,0.168792,0.091086,0.033705,0.168725,...,0.084517,0.104991,0.122254,-0.185707,0.098937,-0.139805,0.051842,0.174006,-0.102040,-0.038772
3,-0.163451,0.166173,-0.217055,-0.132787,0.146147,-0.018958,0.108244,0.084258,0.112765,0.154815,...,0.103792,-0.002166,0.214415,-0.201033,0.050581,-0.172929,0.009328,0.121649,-0.008322,-0.022489
4,-0.156886,0.209307,-0.138229,-0.155249,0.052539,-0.011310,-0.089032,0.000601,0.209711,0.020200,...,0.123560,-0.138099,0.225182,-0.143550,-0.032194,-0.185464,-0.029572,-0.008381,0.107203,0.024711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14797,-0.080916,0.051540,-0.143885,-0.046558,0.127785,0.009149,0.170222,0.080033,0.029520,0.167450,...,0.087989,0.106912,0.125674,-0.179661,0.102458,-0.132932,0.051549,0.166962,-0.089770,-0.043504
14798,-0.150551,0.153922,-0.230572,-0.104825,0.167101,-0.009677,0.126542,0.085814,0.096178,0.170355,...,0.088889,0.028493,0.193977,-0.194510,0.055641,-0.168123,0.024387,0.138275,-0.017380,-0.029095
14799,-0.168736,0.210577,-0.176805,-0.158199,0.088862,-0.011941,-0.053514,0.027774,0.190569,0.055452,...,0.101245,-0.120822,0.224786,-0.156613,-0.016351,-0.181219,-0.024164,0.017461,0.092095,0.017976
14800,-0.160062,0.168297,-0.227097,-0.118343,0.168273,-0.011335,0.107257,0.082741,0.117168,0.169759,...,0.091827,0.017292,0.198628,-0.191600,0.049285,-0.170239,0.019713,0.130282,-0.004818,-0.020066


# Making IsolationForest for content

In [16]:
emb_train_cont['name'] = ex_train['content'].values
emb_train_cont.set_index('name', drop=True, inplace=True)

In [17]:
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(emb_train_cont)

anomaly_labels = model.predict(emb_train_cont)

In [18]:
from collections import Counter #количество аномалий
Counter(anomaly_labels)

Counter({1: 14128, -1: 674})

In [19]:
emb_train_cont['anomaly'] = anomaly_labels

emb_train_cont.columns = emb_train_cont.columns.astype(str)

tsne = TSNE(n_components=2, random_state=42, perplexity=45)
X_2d = tsne.fit_transform(emb_train_cont.drop('anomaly', axis=1))

In [20]:
fig = go.Figure(data=go.Scatter(
    x = X_2d[:, 0],
    y = X_2d[:, 1],
    mode='markers',  # Линии и маркеры
    text=ex_train['content'].values,  # Подписи
    marker=dict(
        size=8,
        color=['red' if anomaly == -1 else 'blue' for anomaly in anomaly_labels],  # Красные для аномалий, синие для обычных точек
    )
))

fig.update_traces(textposition='top center')

fig.show()

In [106]:
with open('/kaggle/working/labels.pkl', 'wb') as file:
    pickle.dump(anomaly_labels, file)
    
with open('/kaggle/working/X_2d.pkl', 'wb') as file:
    pickle.dump(X_2d, file)

In [22]:
emb_train_cont.reset_index(inplace=True)
stream = emb_train_cont[['name', 'anomaly']]

anomaly = stream.loc[stream['anomaly'] == -1]
ar = anomaly['name'].unique()
ar

array(['Система26', 'Письмо Random9838Заявка', 'Письмо Random9439Заявка',
       'Система41', 'Письмо Random2459Заявка', 'Письмо Random622Заявка',
       ' Система8', 'ЭД_ ПЭВМ', 'Система20', 'Система58',
       'Сервисы SNS#1 - Расширенные сервисы', ' 4эт', 'Письмо RandomKKKK',
       'Письмо Random1969Заявка', 'Система22', 'Письмо Random3003Заявка',
       'Письмо Random4834Заявка', 'Письмо Random7318Заявка',
       'Письмо Random9161Заявка', 'Письмо Random314Заявка',
       'Письмо Random8477Заявка', 'Письмо Random4438Заявка',
       '.PDF документы не печатаются', '3. Настройка BIOS',
       'Письмо Random9997Заявка', 'Письмо Random9908Заявка',
       'Письмо Random6358Заявка', 'Письмо Random6968Заявка', ' 1307 вода',
       'Система22.', 'Часы.', 'Письмо Random2487Заявка',
       '*****************', 'Письмо Random4687Заявка',
       'Письмо Random4945Заявка', 'Письмо Random5434Заявка',
       'Письмо Random9104Заявка', 'Письмо Random560Заявка',
       'Письмо Random7546Заявка', '